<a href="https://colab.research.google.com/github/gizdatalab/CPU/blob/main/classifier/SECTOR/CPU_SECTOR_ClimateBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
! pip install datasets transformers sentencepiece huggingface_hub
! apt install git-lfs
! pip install sentence-transformers
! pip install optuna
! pip install evaluate
! pip install accelerate -U
! pip install codecarbon

In [ ]:
from codecarbon import EmissionsTracker

tracker = EmissionsTracker()
tracker.start()

[codecarbon INFO @ 18:52:52] [setup] RAM Tracking...
[codecarbon INFO @ 18:52:52] [setup] GPU Tracking...
[codecarbon INFO @ 18:52:52] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 18:52:52] [setup] CPU Tracking...
[codecarbon WARNING @ 18:52:52] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 18:52:53] We saw that you have a AMD EPYC 7V12 64-Core Processor but we don't know it. Please contact us.
[codecarbon INFO @ 18:52:53] CPU Model on constant consumption mode: AMD EPYC 7V12 64-Core Processor
[codecarbon INFO @ 18:52:53] >>> Tracker's metadata:
[codecarbon INFO @ 18:52:53]   Platform system: Linux-5.15.0-1040-azure-x86_64-with-glibc2.10
[codecarbon INFO @ 18:52:53]   Python version: 3.8.5
[codecarbon INFO @ 18:52:53]   CodeCarbon version: 2.3.2
[codecarbon INFO @ 18:52:53]   Available RAM : 27.406 GB
[codecarbon INFO @ 18:52:53]   CPU count: 4
[codecarbon INFO @ 18:52:53]   CPU model: AMD EPYC 7V12 64-Core Processor
[codecarbon INFO @ 18:52:

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

No such comm: 30f6722c764c4f53afb3d7d3892497ab
No such comm: 04623b6a794c448582e3b4d91748c9d8


In [ ]:
from datasets import load_dataset
import datasets
import numpy as np
import pandas as pd
import evaluate
import torch
import os
import sklearn.metrics as skm
from transformers import (AutoTokenizer, AutoConfig, AutoModelForSequenceClassification,
                          PreTrainedModel, BertModel, BertForSequenceClassification,
                          RobertaForSequenceClassification,
                          TrainingArguments, Trainer, TrainerCallback)
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

2024-01-21 20:18:44.007161: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-21 20:18:44.122111: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-01-21 20:18:44.122132: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-01-21 20:18:44.748683: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-

In [ ]:
# Load Dataset from Hugging Face
try:
    dataset = load_dataset("GIZ/policy_classification", data_files="policy_classification.json",use_auth_token=True )
    data = pd.DataFrame(dataset['train'])
except:
    from huggingface_hub import hf_hub_download
    hf_hub_download("GIZ/policy_classification", filename="policy_classification.json", repo_type="dataset",use_auth_token=True,
    local_dir= 'data/')
    data = pd.read_json('data/policy_classification.json')
data = data.dropna(subset = ['SectorLabel']).reset_index(drop=True)
sector_list = [val  for list_ in list(data.SectorLabel) for val in list_]
sector_list = list(set(sector_list))

for sector in sector_list:
    data[sector] = data.SectorLabel.apply(lambda x: True if sector in x else False)

Using custom data configuration GIZ--policy_classification-cdc4b717b1bd720b


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Failed to read file '/home/azureuser/.cache/huggingface/datasets/downloads/dee24a3afc63cd87c4a0a9517165e43822030f22df905fa089a2628e8d786ca4' with error <class 'pyarrow.lib.ArrowInvalid'>: JSON parse error: Column() changed from object to array in row 0


# Manual Inputs


In [ ]:
###### Define Path to data ############################
repo_id = "GIZ/policy_classification"
filename="policy_classification.json"


### Define labels ##################
label_names= sector_list


#######  Define Model params ######################

# define the model checkpoint
model_checkpoint = "climatebert/distilroberta-base-climate-f"

# define the problem type
problem_type="multi_label_classification"


######## Training Arguments ##################
output_dir = "SECTOR-multilabel-climatebert_w"

# logging and evaluation
evaluation_strategy = "epoch"
logging_strategy="epoch"
logging_steps = 1
eval_steps = 1

# Important params
learning_rate=9.07e-5   #4.04e-5
batch_size = 8
num_train_epochs= 10
weight_decay=0.004
warmup_steps = 300
gradient_accumulation_steps = 2

# Automated

## Dataset

In [ ]:
# Load Dataset from Hugging Face
# Load Dataset from Hugging Face
# try:
#     dataset = load_dataset(repo_id, data_files=filename,
#                              use_auth_token=True )
#     data = pd.DataFrame(dataset['train'])
# except:
#     from huggingface_hub import hf_hub_download
#     hf_hub_download(repo_id, filename=filename,
#                     repo_type="dataset",use_auth_token=True,
#     local_dir= 'data/')
#     data = pd.read_json('data/policy_classification.json')

# the dataset already contains train_test split flag we use
# it to create train-test split.

train_df = data[data.split== 'Train'].reset_index(drop=True)
test_df = data[data.split=='Test'].reset_index(drop=True)

# Sort the labels and create plaecholders for id2label and label2id
label_names.sort()

num_labels = len(label_names)
id2label = {idx:label for idx, label in enumerate(label_names)}
label2id = {label:idx for idx, label in enumerate(label_names)}
print(id2label)

{0: 'Agriculture', 1: 'Buildings', 2: 'Coastal Zone', 3: 'Cross-Cutting Area', 4: 'Disaster Risk Management (DRM)', 5: 'Economy-wide', 6: 'Education', 7: 'Energy', 8: 'Environment', 9: 'Health', 10: 'Industries', 11: 'LULUCF/Forestry', 12: 'Social Development', 13: 'Tourism', 14: 'Transport', 15: 'Urban', 16: 'Waste', 17: 'Water'}


In [ ]:
def prepare_multilabel_df(df, cols):
    """
    Takes the dataframe and list of columns which need to combined into
    One-hot-encoded labels.

    """
    df.dropna(subset = cols, inplace=True)
    df = df.reset_index(drop=True)
    df['labels'] = df.apply(lambda x: [int(x[label]) for label in cols],axis=1)
    # we have multilingual dataset ('en', 'fr','es'), in case of non-english the
    # translated Context is udner the column 'TranslatedContext'. We impute these
    # back into 'context' to work with only English dataset.
    df['Context'] = df.apply(lambda x: x['Context'] if x['Language'] == 'en'
                                    else x['TranslatedContext'][0],axis=1)
    df.rename(columns = {'Context':'text'}, inplace=True)

    df = df[['text','labels']+cols]
    return df

# creating multilabel dataset
train_df = prepare_multilabel_df(train_df, label_names)
test_df = prepare_multilabel_df(test_df, label_names)

print('\n','Classes Representation in Training Dataset:',len(train_df))
for i,label in enumerate(label_names):
    print(i,".",label, ":", sum( [x[i] for x in list(train_df['labels'])]))

print('\n','Classes Representation in Test Dataset:',len(test_df))
for i,label in enumerate(label_names):
    print(i,".",label, ":", sum( [x[i] for x in list(test_df['labels'])]))


 Classes Representation in Training Dataset: 10123
0 . Agriculture : 2235
1 . Buildings : 169
2 . Coastal Zone : 698
3 . Cross-Cutting Area : 1853
4 . Disaster Risk Management (DRM) : 814
5 . Economy-wide : 873
6 . Education : 180
7 . Energy : 2847
8 . Environment : 905
9 . Health : 662
10 . Industries : 419
11 . LULUCF/Forestry : 1861
12 . Social Development : 507
13 . Tourism : 192
14 . Transport : 1173
15 . Urban : 558
16 . Waste : 714
17 . Water : 1207

 Classes Representation in Test Dataset: 1125
0 . Agriculture : 236
1 . Buildings : 21
2 . Coastal Zone : 78
3 . Cross-Cutting Area : 204
4 . Disaster Risk Management (DRM) : 92
5 . Economy-wide : 100
6 . Education : 27
7 . Energy : 315
8 . Environment : 95
9 . Health : 76
10 . Industries : 51
11 . LULUCF/Forestry : 215
12 . Social Development : 70
13 . Tourism : 28
14 . Transport : 133
15 . Urban : 61
16 . Waste : 73
17 . Water : 124


In [ ]:
for i,label in enumerate(label_names):
    # df_train[sector] = df_train.apply(lambda x: x['sector_label'][i], axis =1)
    print(i,".",label, ":", sum(train_df[label]))

# these are weights which can be used in building weighted loss if class is imbalanced
positive_weights = {}
negative_weights = {}
for label in label_names:
    positive_weights[label] = train_df.shape[0]/(2*np.count_nonzero(train_df[label]==True))
    negative_weights[label] = train_df.shape[0]/(2*np.count_nonzero(train_df[label]==False))
print("Positive Weights",'\n',positive_weights)
print("Negative Weights",'\n',negative_weights)

# as we dont want to miss anything (high recall) we use positive weights calculated above.
pos_weights = list(positive_weights.values())

# If using GPU we need to place all required data on else there will be error
posweights = torch.FloatTensor(pos_weights).to(device)

0 . Agriculture : 2235
1 . Buildings : 169
2 . Coastal Zone : 698
3 . Cross-Cutting Area : 1853
4 . Disaster Risk Management (DRM) : 814
5 . Economy-wide : 873
6 . Education : 180
7 . Energy : 2847
8 . Environment : 905
9 . Health : 662
10 . Industries : 419
11 . LULUCF/Forestry : 1861
12 . Social Development : 507
13 . Tourism : 192
14 . Transport : 1173
15 . Urban : 558
16 . Waste : 714
17 . Water : 1207
Positive Weights 
 {'Agriculture': 2.264653243847875, 'Buildings': 29.949704142011836, 'Coastal Zone': 7.251432664756447, 'Cross-Cutting Area': 2.731516459794927, 'Disaster Risk Management (DRM)': 6.218058968058968, 'Economy-wide': 5.797823596792669, 'Education': 28.119444444444444, 'Energy': 1.7778363189322093, 'Environment': 5.592817679558011, 'Health': 7.645770392749244, 'Industries': 12.079952267303103, 'LULUCF/Forestry': 2.7197743148844706, 'Social Development': 9.983234714003945, 'Tourism': 26.361979166666668, 'Transport': 4.315004262574595, 'Urban': 9.07078853046595, 'Waste': 

`pos_weight > 1 will increase the recall while pos_weight < 1 will increase the precision.`

## Model

In [ ]:
# problem_type, is not needed in tokenizer but keeping it for conformity
# https://huggingface.co/docs/transformers/main_classes/configuration?highlight=multi_label_classification#transformers.PretrainedConfig
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint,problem_type=problem_type)

# Set the model config and assign it and device to model
config = AutoConfig.from_pretrained(model_checkpoint, num_labels=num_labels, id2label=id2label, label2id=label2id, problem_type=problem_type)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint,config=config).to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at climatebert/distilroberta-base-climate-f and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# we need datasets format to work with
train_ds = datasets.Dataset.from_pandas(train_df[['text','labels']])
test_ds =  datasets.Dataset.from_pandas(test_df[['text','labels']])
train_ds = train_ds.shuffle(seed=7)
test_ds = test_ds.shuffle(seed=7)


# Need to tokenize the data using the tokenizer of the model
def tokenize_and_encode(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True,
                        max_length=384)

cols = train_ds.column_names
cols.remove("labels")
print('Training data:',train_ds.num_rows)
print('Validation data:',test_ds.num_rows)

train_tokenized = train_ds.map(tokenize_and_encode, batched=True, remove_columns= cols)
val_tokenized = test_ds.map(tokenize_and_encode, batched=True, remove_columns= cols)

# need this to avoid error due to type mismatch
# https://discuss.pytorch.org/t/multi-label-binary-classification-result-type-float-cant-be-cast-to-the-desired-output-type-long/117915/3
train_tokenized.set_format("torch")
train_tokenized = (train_tokenized
          .map(lambda x : {"float_labels": x["labels"].to(torch.float)}, remove_columns=["labels"])
          .rename_column("float_labels", "labels"))

val_tokenized.set_format("torch")
val_tokenized = (val_tokenized
          .map(lambda x : {"float_labels": x["labels"].to(torch.float)}, remove_columns=["labels"])
          .rename_column("float_labels", "labels"))

Parameter 'function'=<function tokenize_and_encode at 0x7fc8910ca8b0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Training data: 10123
Validation data: 1125


  0%|          | 0/11 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/10123 [00:00<?, ?ex/s]

  0%|          | 0/1125 [00:00<?, ?ex/s]

## Metrics

In [ ]:
multilabel_f1_metric = evaluate.load("f1", "multilabel")
multilabel_recall_metric = evaluate.load("recall","multilabel")
multilabel_precision_metric = evaluate.load("precision","multilabel")

def get_scores(y_pred, y_test, thresh=0.5, sigmoid=True):
    """
    Takes the Predictions and Truth Values, and return multilabel metrics
    aggregated on multiple strategies['micro', 'sample', 'weighted']

    """
    # we get raw function value, therefore we apply sigmoid to get to the
    # probabilities
    if problem_type == "multi_label_classification":
        y_pred = torch.from_numpy(y_pred)
        y_test = torch.from_numpy(y_test)
        if sigmoid:
          y_pred = y_pred.sigmoid()
          y_pred = (y_pred > thresh)

        return {
            "precision-micro": multilabel_precision_metric.compute(predictions=y_pred, references=y_test, average="micro")["precision"],
            "precision-samples": multilabel_precision_metric.compute(predictions=y_pred, references=y_test, average="samples")["precision"],
            "precision-weighted": multilabel_precision_metric.compute(predictions=y_pred, references=y_test, average="weighted")["precision"],
            "recall-micro": multilabel_recall_metric.compute(predictions=y_pred, references=y_test, average="micro")["recall"],
            "recall-samples": multilabel_recall_metric.compute(predictions=y_pred, references=y_test, average="samples")["recall"],
            "recall-weighted": multilabel_recall_metric.compute(predictions=y_pred, references=y_test, average="weighted")["recall"],
            "f1-micro": multilabel_f1_metric.compute(predictions=y_pred, references=y_test, average="micro")["f1"],
            "f1-samples": multilabel_f1_metric.compute(predictions=y_pred, references=y_test, average="samples")["f1"],
            "f1-weighted": multilabel_f1_metric.compute(predictions=y_pred, references=y_test, average="weighted")["f1"],
            }
    else:
        y_pred = torch.from_numpy(y_pred).argmax(dim=1)
        y_test = torch.from_numpy(y_test)

        report = skm.classification_report(y_test, y_pred, output_dict=True)
        df_report = pd.DataFrame(report).transpose()
        return {"Precision_macro": df_report.loc['macro avg']['precision'],
                "Precision_weighted": df_report.loc['weighted avg']['precision'],
                "Recall_macro": df_report.loc['macro avg']['recall'],
                "Recall_weighted": df_report.loc['weighted avg']['recall'],
                "F1-Score": df_report.loc['macro avg']['f1-score'],
                "Accuracy": skm.accuracy_score(y_test, y_pred)}


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    return get_scores(predictions, labels)

## Training arguments

In [ ]:
args = TrainingArguments(
    output_dir,
    logging_strategy=logging_strategy,
    evaluation_strategy = evaluation_strategy,
    save_strategy = "epoch",
    logging_steps = logging_steps,
    eval_steps= eval_steps,
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    weight_decay=weight_decay,
    lr_scheduler_type = "linear",
    gradient_accumulation_steps = gradient_accumulation_steps,
    warmup_steps = warmup_steps,
    push_to_hub = True,
)

## Custom Trainer

In [ ]:
# for class weights we need to use Custom Multi-Label Trainer
# In multi-label problem we will be using Binary Cross Entropy loss with
# sigmoid layer on top rather than softmax.
class MultilabelTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fct = torch.nn.BCEWithLogitsLoss(pos_weight=posweights)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels),
                        labels.float().view(-1, self.model.config.num_labels))
        return (loss, outputs) if return_outputs else loss


# Training

In [ ]:
multi_trainer =  Trainer(
    model,
    args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer)


In [ ]:
multi_trainer.train()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
You're using a RobertaTokeni

Epoch,Training Loss,Validation Loss,Precision-micro,Precision-samples,Precision-weighted,Recall-micro,Recall-samples,Recall-weighted,F1-micro,F1-samples,F1-weighted
1,0.235500,0.182517,0.773312,0.727926,0.757461,0.481241,0.620402,0.481241,0.593278,0.634657,0.545446
2,0.159000,0.165120,0.781250,0.760519,0.765566,0.525263,0.666326,0.525263,0.628178,0.676268,0.603080
3,0.128300,0.154105,0.772843,0.785042,0.761438,0.609305,0.725400,0.609305,0.681399,0.724153,0.668919
4,0.100600,0.156900,0.775155,0.809366,0.768878,0.624312,0.755912,0.624312,0.691604,0.748301,0.674826
5,0.076700,0.153681,0.744486,0.804084,0.738701,0.692346,0.797707,0.692346,0.717470,0.768121,0.710420
6,0.059000,0.157696,0.758260,0.821976,0.754771,0.688844,0.795717,0.688844,0.721887,0.777550,0.716969
7,0.043600,0.160372,0.755102,0.816194,0.757575,0.703352,0.805851,0.703352,0.728309,0.779613,0.727044
8,0.032800,0.164796,0.751176,0.822717,0.746313,0.718859,0.821746,0.718859,0.734663,0.789911,0.729588
9,0.024300,0.162662,0.764923,0.830194,0.761259,0.724362,0.822704,0.724362,0.744090,0.795679,0.741037
10,0.019900,0.165071,0.766082,0.827886,0.762021,0.720860,0.820898,0.720860,0.742784,0.793901,0.738650


[codecarbon INFO @ 18:54:08] Energy consumed for RAM : 0.000214 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 18:54:08] Energy consumed for all GPUs : 0.000603 kWh. Total GPU Power : 42.15495270268354 W
[codecarbon INFO @ 18:54:08] Energy consumed for all CPUs : 0.000885 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 18:54:08] 0.001702 kWh of electricity used since the beginning.
[codecarbon INFO @ 18:54:17] Energy consumed for RAM : 0.000043 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 18:54:17] Energy consumed for all GPUs : 0.000286 kWh. Total GPU Power : 68.6115671439928 W
[codecarbon INFO @ 18:54:17] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 18:54:17] 0.000506 kWh of electricity used since the beginning.
[codecarbon INFO @ 18:54:23] Energy consumed for RAM : 0.000257 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 18:54:23] Energy consumed for all GPUs : 0.000891 kWh. Total GPU Power : 69.29745838453462 W
[

TrainOutput(global_step=6330, training_loss=0.08797564363404285, metrics={'train_runtime': 3910.0332, 'train_samples_per_second': 25.89, 'train_steps_per_second': 1.619, 'total_flos': 1.006012578461184e+16, 'train_loss': 0.08797564363404285, 'epoch': 10.0})

[codecarbon INFO @ 20:00:40] Energy consumed for RAM : 0.011605 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 20:00:40] Energy consumed for all GPUs : 0.072020 kWh. Total GPU Power : 26.765033743787637 W
[codecarbon INFO @ 20:00:40] Energy consumed for all CPUs : 0.048008 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 20:00:40] 0.131633 kWh of electricity used since the beginning.
[codecarbon INFO @ 20:00:55] Energy consumed for RAM : 0.011648 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 20:00:55] Energy consumed for all GPUs : 0.072132 kWh. Total GPU Power : 26.80238303782647 W
[codecarbon INFO @ 20:00:55] Energy consumed for all CPUs : 0.048185 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 20:00:55] 0.131965 kWh of electricity used since the beginning.
[codecarbon INFO @ 20:01:10] Energy consumed for RAM : 0.011691 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 20:01:10] Energy consumed for all GPUs : 0.072242 kWh. Total GPU Power : 26.46847627333877 W

In [ ]:
log_results = multi_trainer.state.log_history
log_results

[{'loss': 0.722,
  'learning_rate': 4.4163923076923076e-05,
  'epoch': 1.0,
  'step': 633},
 {'eval_loss': 0.5413973331451416,
  'eval_precision-micro': 0.46779336734693877,
  'eval_precision-samples': 0.5878356581689914,
  'eval_precision-weighted': 0.5486701714956371,
  'eval_recall-micro': 0.7338669334667334,
  'eval_recall-samples': 0.8284062184728852,
  'eval_recall-weighted': 0.7338669334667334,
  'eval_f1-micro': 0.5713729308666017,
  'eval_f1-samples': 0.633307325209286,
  'eval_f1-weighted': 0.6039257217949973,
  'eval_runtime': 13.1365,
  'eval_samples_per_second': 85.639,
  'eval_steps_per_second': 10.733,
  'epoch': 1.0,
  'step': 633},
 {'loss': 0.4903,
  'learning_rate': 8.832784615384615e-05,
  'epoch': 2.0,
  'step': 1266},
 {'eval_loss': 0.5138183832168579,
  'eval_precision-micro': 0.49881154499151104,
  'eval_precision-samples': 0.6188608946608946,
  'eval_precision-weighted': 0.5639108552453644,
  'eval_recall-micro': 0.7348674337168585,
  'eval_recall-samples': 0.8

Classweight Training

In [ ]:
multi_trainer =  MultilabelTrainer(
    model,
    args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
[codecarbon INFO @ 20:20:58] [setup] RAM Tracking...
[codecarbon INFO @ 20:20:58] [setup] GPU Tracking...
[codecarbon INFO @ 20:20:58] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 20:20:58] [setup] CPU Tracking...
[codecarbon WARNING @ 20:20:58] No CPU tracking mode found. Falling back on CPU constant mode.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
[codecarbon WARNING @ 20:20:59] We saw

In [ ]:
multi_trainer.train()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
You're using a RobertaTokeni

Epoch,Training Loss,Validation Loss,Precision-micro,Precision-samples,Precision-weighted,Recall-micro,Recall-samples,Recall-weighted,F1-micro,F1-samples,F1-weighted
1,0.722000,0.541397,0.467793,0.587836,0.548670,0.733867,0.828406,0.733867,0.571373,0.633307,0.603926
2,0.490300,0.513818,0.498812,0.618861,0.563911,0.734867,0.827348,0.734867,0.594256,0.659675,0.618515
3,0.408000,0.460262,0.568545,0.692352,0.618061,0.746873,0.836198,0.746873,0.645622,0.714071,0.663741
4,0.304500,0.430582,0.574692,0.706081,0.604739,0.792896,0.871180,0.792896,0.666386,0.736716,0.678964
5,0.222300,0.432378,0.586925,0.728869,0.618497,0.817409,0.884680,0.817409,0.683253,0.756782,0.696391
6,0.160700,0.466620,0.625484,0.750108,0.646031,0.807904,0.879560,0.807904,0.705086,0.769590,0.712863
7,0.116700,0.496069,0.696025,0.786071,0.710533,0.770885,0.850918,0.770885,0.731545,0.783092,0.736130
8,0.082800,0.548422,0.720465,0.804911,0.723736,0.774887,0.859757,0.774887,0.746686,0.797853,0.746409
9,0.059600,0.550595,0.729351,0.808380,0.732108,0.781891,0.861866,0.781891,0.754708,0.801794,0.755071
10,0.045200,0.564022,0.729566,0.812203,0.732844,0.781391,0.864086,0.781391,0.754589,0.804535,0.754998


[codecarbon INFO @ 20:21:19] Energy consumed for RAM : 0.000043 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 20:21:19] Energy consumed for all GPUs : 0.000286 kWh. Total GPU Power : 68.70070522549551 W
[codecarbon INFO @ 20:21:19] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 20:21:19] 0.000506 kWh of electricity used since the beginning.
[codecarbon INFO @ 20:21:34] Energy consumed for RAM : 0.000086 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 20:21:34] Energy consumed for all GPUs : 0.000576 kWh. Total GPU Power : 69.50065331190542 W
[codecarbon INFO @ 20:21:34] Energy consumed for all CPUs : 0.000354 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 20:21:34] 0.001016 kWh of electricity used since the beginning.
[codecarbon INFO @ 20:21:49] Energy consumed for RAM : 0.000128 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 20:21:49] Energy consumed for all GPUs : 0.000865 kWh. Total GPU Power : 69.42505174613022 W


TrainOutput(global_step=6330, training_loss=0.2612001602871911, metrics={'train_runtime': 4027.2082, 'train_samples_per_second': 25.137, 'train_steps_per_second': 1.572, 'total_flos': 1.006012578461184e+16, 'train_loss': 0.2612001602871911, 'epoch': 10.0})

In [ ]:
# class weight
log_results = multi_trainer.state.log_history
log_results

[{'loss': 0.722,
  'learning_rate': 4.4163923076923076e-05,
  'epoch': 1.0,
  'step': 633},
 {'eval_loss': 0.5413973331451416,
  'eval_precision-micro': 0.46779336734693877,
  'eval_precision-samples': 0.5878356581689914,
  'eval_precision-weighted': 0.5486701714956371,
  'eval_recall-micro': 0.7338669334667334,
  'eval_recall-samples': 0.8284062184728852,
  'eval_recall-weighted': 0.7338669334667334,
  'eval_f1-micro': 0.5713729308666017,
  'eval_f1-samples': 0.633307325209286,
  'eval_f1-weighted': 0.6039257217949973,
  'eval_runtime': 13.1365,
  'eval_samples_per_second': 85.639,
  'eval_steps_per_second': 10.733,
  'epoch': 1.0,
  'step': 633},
 {'loss': 0.4903,
  'learning_rate': 8.832784615384615e-05,
  'epoch': 2.0,
  'step': 1266},
 {'eval_loss': 0.5138183832168579,
  'eval_precision-micro': 0.49881154499151104,
  'eval_precision-samples': 0.6188608946608946,
  'eval_precision-weighted': 0.5639108552453644,
  'eval_recall-micro': 0.7348674337168585,
  'eval_recall-samples': 0.8

In [ ]:
tracker.stop()

[codecarbon INFO @ 20:02:38] Energy consumed for RAM : 0.011943 kWh. RAM Power : 10.2771635055542 W
[codecarbon INFO @ 20:02:38] Energy consumed for all GPUs : 0.072885 kWh. Total GPU Power : 25.95743242760474 W
[codecarbon INFO @ 20:02:38] Energy consumed for all CPUs : 0.049404 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 20:02:38] 0.134232 kWh of electricity used since the beginning.


0.049058572731587105

In [ ]:
multi_trainer.push_to_hub()

'https://huggingface.co/ppsingh/SECTOR-multilabel-climatebert_w/tree/main/'

In [ ]:
import json
with open("classifier_info/sector_climatebert_logs_w.json", "w") as fp:
    json.dump(log_results , fp)

# Predict and Evaluate

## Trainer

In [ ]:
predictions= multi_trainer.predict(val_tokenized)
pred,labels,_ = predictions
y_pred = torch.from_numpy(pred)
y_true = torch.from_numpy(labels)
y_prob = y_pred.sigmoid()
thresh = 0.5
y_pred = (y_prob>=thresh).bool()

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
cm = skm.multilabel_confusion_matrix(y_true, y_pred)
for i,label in enumerate(label_names):
    print(label)
    print(cm[i],'\n')

Agriculture
[[845  44]
 [ 37 199]] 

Buildings
[[1098    6]
 [  10   11]] 

Coastal Zone
[[1030   17]
 [  25   53]] 

Cross-Cutting Area
[[867  54]
 [ 73 131]] 

Disaster Risk Management (DRM)
[[1005   28]
 [  37   55]] 

Economy-wide
[[987  38]
 [ 57  43]] 

Education
[[1095    3]
 [  14   13]] 

Energy
[[767  43]
 [ 46 269]] 

Environment
[[998  32]
 [ 35  60]] 

Health
[[1038   11]
 [  19   57]] 

Industries
[[1065    9]
 [   9   42]] 

LULUCF/Forestry
[[865  45]
 [ 39 176]] 

Social Development
[[1044   11]
 [  30   40]] 

Tourism
[[1092    5]
 [  14   14]] 

Transport
[[967  25]
 [ 34  99]] 

Urban
[[1048   16]
 [  31   30]] 

Waste
[[1033   19]
 [  10   63]] 

Water
[[967  34]
 [ 38  86]] 



In [ ]:
report = skm.classification_report(y_true, y_pred, output_dict=True)
df_report = pd.DataFrame(report).transpose()
df_report = df_report.reset_index()
mapping = {str(i):j for i,j in enumerate(label_names)}
df_report['index'] = df_report['index'].map(mapping).fillna(df_report['index'])
df_report.rename(columns = {'index':'label'}, inplace=True)
df_report

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,label,precision,recall,f1-score,support
0,Agriculture,0.818930,0.843220,0.830898,236.0
1,Buildings,0.647059,0.523810,0.578947,21.0
2,Coastal Zone,0.757143,0.679487,0.716216,78.0
3,Cross-Cutting Area,0.708108,0.642157,0.673522,204.0
4,Disaster Risk Management (DRM),0.662651,0.597826,0.628571,92.0
5,Economy-wide,0.530864,0.430000,0.475138,100.0
6,Education,0.812500,0.481481,0.604651,27.0
7,Energy,0.862179,0.853968,0.858054,315.0
8,Environment,0.652174,0.631579,0.641711,95.0
9,Health,0.838235,0.750000,0.791667,76.0


### Classweight Results

In [ ]:
# class weights
cm = skm.multilabel_confusion_matrix(y_true, y_pred)
for i,label in enumerate(label_names):
    print(label)
    print(cm[i],'\n')

Agriculture
[[839  50]
 [ 30 206]] 

Buildings
[[1093   11]
 [   3   18]] 

Coastal Zone
[[1016   31]
 [  20   58]] 

Cross-Cutting Area
[[846  75]
 [ 54 150]] 

Disaster Risk Management (DRM)
[[1000   33]
 [  29   63]] 

Economy-wide
[[971  54]
 [ 45  55]] 

Education
[[1093    5]
 [   9   18]] 

Energy
[[764  46]
 [ 44 271]] 

Environment
[[986  44]
 [ 28  67]] 

Health
[[1035   14]
 [  17   59]] 

Industries
[[1057   17]
 [   6   45]] 

LULUCF/Forestry
[[851  59]
 [ 36 179]] 

Social Development
[[1040   15]
 [  24   46]] 

Tourism
[[1093    4]
 [   9   19]] 

Transport
[[957  35]
 [ 29 104]] 

Urban
[[1039   25]
 [  23   38]] 

Waste
[[1026   26]
 [   7   66]] 

Water
[[966  35]
 [ 24 100]] 



In [ ]:
# class weights results
report = skm.classification_report(y_true, y_pred, output_dict=True)
df_report = pd.DataFrame(report).transpose()
df_report = df_report.reset_index()
mapping = {str(i):j for i,j in enumerate(label_names)}
df_report['index'] = df_report['index'].map(mapping).fillna(df_report['index'])
df_report.rename(columns = {'index':'label'}, inplace=True)
df_report

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,label,precision,recall,f1-score,support
0,Agriculture,0.804688,0.872881,0.837398,236.0
1,Buildings,0.620690,0.857143,0.720000,21.0
2,Coastal Zone,0.651685,0.743590,0.694611,78.0
3,Cross-Cutting Area,0.666667,0.735294,0.699301,204.0
4,Disaster Risk Management (DRM),0.656250,0.684783,0.670213,92.0
5,Economy-wide,0.504587,0.550000,0.526316,100.0
6,Education,0.782609,0.666667,0.720000,27.0
7,Energy,0.854890,0.860317,0.857595,315.0
8,Environment,0.603604,0.705263,0.650485,95.0
9,Health,0.808219,0.776316,0.791946,76.0


## Pipeline

In [ ]:
from transformers import pipeline
model_checkpoint = "ppsingh/TAPP-multilabel-climatebert"
pipe = pipeline("text-classification", model=model_checkpoint, top_k=None, device=device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
predictions = pipe(list(test_df['text']))
pred = []
for prediction in predictions:
    pred.append({x['label']:x['score'] for x in prediction})
df_pred = pd.DataFrame(pred)
df_pred['labels'] = df_pred.apply(lambda x: np.array([x[label]
                            for label in label_names]) >= 0.50,axis=1)
y_true = np.array(list(test_df['labels']))
y_pred = np.array(list(df_pred['labels']))

In [ ]:
cm = skm.multilabel_confusion_matrix(y_true, y_pred)
for i,label in enumerate(label_names):
    print(label)
    print(cm[i],'\n')

ActionLabel
[[417 104]
 [ 69 531]] 

PlansLabel
[[807  77]
 [ 71 166]] 

PolicyLabel
[[939  30]
 [ 28 124]] 

TargetLabel
[[737  62]
 [ 37 285]] 



In [ ]:
report = skm.classification_report(y_true, y_pred, output_dict=True)
df_report = pd.DataFrame(report).transpose()
df_report = df_report.reset_index()
mapping = {str(i):j for i,j in enumerate(label_names)}
df_report['index'] = df_report['index'].map(mapping).fillna(df_report['index'])
df_report.rename(columns = {'index':'label'}, inplace=True)
df_report

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,label,precision,recall,f1-score,support
0,ActionLabel,0.836220,0.885000,0.859919,600.0
1,PlansLabel,0.683128,0.700422,0.691667,237.0
2,PolicyLabel,0.805195,0.815789,0.810458,152.0
3,TargetLabel,0.821326,0.885093,0.852018,322.0
4,micro avg,0.802030,0.843631,0.822305,1311.0
5,macro avg,0.786467,0.821576,0.803515,1311.0
6,weighted avg,0.801289,0.843631,0.821827,1311.0
7,samples avg,0.787021,0.808802,0.782698,1311.0
